# Review of Probability

## CSCI E-83
## Stephen Elston

Probability theory is at the core of probabilistic programming. Therefore, it is important to have a good understanding of the principles of probability theory. This lesson introduces you to the basic concepts you will need to know to tackle probabilistic programming. Specifically you will learn:

1. The three axioms of probability.
2. How to work with conditional probability and Bayes' rule. 
3. Factoring probability distributions using the chain rule of probability. 
3. Computing marginal distributions for inference. 
4. Applying the concepts of dependence and independence to factor distributions. 

As a first step run the code below to import the required packages. 

In [1]:
import pandas as pd
import numpy as np

## Axioms of probability

All probability distributions must have a certain properties, which we refer to as the **axioms of probability**. 

A discrete distribution is defined by its **probability mass** function, with nonzero values as discrete points. We can speak of a **set of events** within the **sample space** of all possible events. For discrete distributions the axioms then become:

- Probability for any set, A, is bounded between 0 and 1:  

$$0 \le P(A) \le 1 $$
- For a discrete distribution, the sum of the probability over the Sample Space = 1:  

$$P(S) = \sum_{All\ i}P(a_i) = 1 $$

- The probability of finite independent unions is the sum of their probabilities:

$$P(A \cup B) = P(A) + P(B)\\ 
if\ and\ only\ if\\ 
A \cap B = 0 $$

In contrast to discrete distributions, a **continuous distribution** is defined by its **probability density** function. For a continuous distribution the first and second axioms become:

$$0 \le P(a, b) = \int_a^b P(x) dx  \le 1$$

and 

$$P(-\infty, \infty) = \int_{-\infty}^\infty P(x) dx  = 1$$

As a corollary of the first axiom, you can see that the probability over and infinitesimal interval must be zero:

$$0 \le P(a, a) = \int_a^a P(x) dx  = 0$$

To make these ideas concrete, let's try an example. The code in the cell below creates a data frame with the the probabilities of hair and eye color combinations. 

In [2]:
eyeHair = pd.DataFrame({'Black':[0.11, 0.03, 0.03, 0.01], 
                     'Brunette':[0.21, 0.14, 0.09, 0.05],
                     'Red':[0.04, 0.03, 0.02, 0.02],
                     'Blond':[0.01, 0.16, 0.02, 0.03]}, 
                      index = ['Brown', 'Blue', 'Hazel', 'Green'])
eyeHair

,Black,Blond,Brunette,Red
Brown,0.11,0.01,0.21,0.04
Blue,0.03,0.16,0.14,0.03
Hazel,0.03,0.02,0.09,0.02
Green,0.01,0.03,0.05,0.02


This table contains a bivariate **joint distribution** of $p(hair,eye)$. By joint distribution we mean a probability distribution that depends on multiple variables; two in this case. For example, the probability of a subject in this sample having black hair and brown eyes: $p(black,brown) = 0.11$ .

You can see that all of these probabilities are in the range $0 \le p(hair,eye) \le 1.0$, and therefore satisfy one of the axioms. 

We can test if these probabilities add up to 1.0. 

In [3]:
np.array(eyeHair).sum()

0.9999999999999999

To within the rounding error, this probabilities add to 1.0 and satisfy another axiom. 

The question of independence or dependence is a bit more complicated, and will be addressed later. 

## Conditional distributions and Bayes' Theorem

A probability distribution of one random variable can be conditionally dependent on another random variable. **Bayes' theorem**, also known as **Bayes' rule**, gives us a powerful tool to think about and analyze conditional probabilities. We can 

$$P(A \cap B) = P(A|B)P(B)\\
P(B \cap A) = P(B|A)P(A)$$

Now:

$$P(A \cap B) = P(B \cap A)$$

This leads to Bayes' theorem as follows:

$$P(A|B)P(B) = P(B|A)P(A)\\
P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

Which is Bayes' theorem! 

How can we interpret Bayes' theorem? We can think of Bayes' theorem in the following terms:

$$Posterior(hypotheis\ |\ evidence) = \frac{Likelihood(evidence\ |\ hypothesis)\ prior(hypothesis)}{\sum_{h' \in\ All\ possible\ hypotheses}Likelihood(evidence\ |\ h')\ prior(h')}$$

In other words, we can find the posterior probability of a hypothesis using Bayes' theorem. Using the likelihood and prior probability of the hypothesis. Notice that the normalization is the sum over all possible hypotheses. This can be problematic in practice.  

We can compute $p(eye | hair = Black)$. First we can find the probabilities from the joint distribution of $p(eye, hair = Black)$:

In [4]:
eyeHair_black = eyeHair.loc[:,'Black']
eyeHair_black

Brown    0.11
Blue     0.03
Hazel    0.03
Green    0.01
Name: Black, dtype: float64

These numbers cannot be a probability distribution as they do not add up to 1.0. However, normalizing is easy in this case. This gives us the **marginal probability distribution** of eye color given black hair:

In [5]:
eyeHair_black = eyeHair_black/sum(eyeHair_black)
eyeHair_black

Brown    0.611111
Blue     0.166667
Hazel    0.166667
Green    0.055556
Name: Black, dtype: float64

The above calculation is an example of **inference**. We have inferred the marginal distribution of eye color given black hair from the joint distribution. 

We can also find the mostly likely or most probable eye color given black hair:

In [6]:
print('For pople with Black eyes: \nMost common hair color = ' + eyeHair_black.idxmax() + 
      '\nwith probability = %4.2f' % (eyeHair_black.max()))

For pople with Black eyes: 
Most common hair color = Brown
with probability = 0.61


## Chain rule of probability

Another way to work with distributions is by applying the **chain rule of probability**. This rule allows us to factor a joint distribution as follows:

$$P(A_1, A_2, A_3, A_4 \ldots, A_n) = P(A_1 | A_2, A_3, A_4, \ldots, A_n)\ P(A_2, A_3, A_4 \ldots, A_n)$$

In words, a joint distribution can be factored as a distribution of one of the variable, conditioned on the other variables, multiplied by the joint distribution of the other variables. 

We can continue this factorization until we reach an end point:

$$P(A_1, A_2, A_3, A_4 \ldots, A_n) = P(A_1 | A_2, A_3, A_4, \ldots, A_n)\ P(A_2 | A_3, A_4 \ldots, A_n)\ P(A_3| A_4 \ldots, A_n) \ldots p(A_n)$$

Or in general terms, we can expand a joint distribution as the product of conditional distributions:

$$P(\bigcap_{k=1}^n A_k) = \prod_{k=1}^n p(A_k \big| \bigcap_{j=1}^{n-1} A_j)$$

> **Note:** The factorization is not unique. We can factor the variables in any order. In fact, for a joint distribution with $n$ variables, there are $n!$ unique factorizations. For example, we can factorize the foregoing distribution as:

$$P(A_1, A_2, A_3, A_4 \ldots, A_n) = P(A_n | A_{n-1}, A_{n-2}, A_{n-3}, \ldots, A_1)\ P(A_{n-1}| A_{n-2}, A_{n-3}, \ldots, A_1)\ P(A_{n-2}| A_{n-3}, \ldots, A_1) \ldots p(A_1)$$

As an example of a factorization using the chain rule of probability, we can find the conditional distribution of eye color given hair color (or the reverse). Our table of data only gives us the joint distributions which we can factor as:

$$P(eye,hair) = P(eye|hair)\ P(hair) \\
or,\\
P(eye|hair) = \frac{P(eye,hair)}{P(hair)}$$

## Marginal distributions

Inference for probabilistic models is typically performed by computing **marginal distributions**. A marginal distribution is the probability distribution of one or more variables summed or integrated over the other variables of a multivariate distribution. This process is often an essential step in performing **inference**. By inference, we mean returning the results of a query on a variable of a probabilistic model. 

For example, if we start with a joint distribution we can factor it using the chain rule of probabilities:

$$p(A,B) = P(A|B)p(B)$$

And, we can compute the marginal distribution over $A$ by summing over $B$:

$$P(A) = \sum_{B} P(A|B)p(B) $$

The concept is simple. The the result of the summation is the distribution on the *margin* of the multivariate distribution. 

Using our dataset of hair and eye color, it is easy to compute the marginal probabilities of eye color as follows:

In [7]:
eyeHair['MarginalEye'] = eyeHair.sum(axis = 1)
eyeHair

,Black,Blond,Brunette,Red,MarginalEye
Brown,0.11,0.01,0.21,0.04,0.37
Blue,0.03,0.16,0.14,0.03,0.36
Hazel,0.03,0.02,0.09,0.02,0.16
Green,0.01,0.03,0.05,0.02,0.11


We can also compute the marginal probability of hair color:

In [8]:
eyeHair = pd.concat([eyeHair, pd.DataFrame({'MarginalHair':eyeHair.sum(axis = 0)}).T])
eyeHair

,Black,Blond,Brunette,Red,MarginalEye
Brown,0.11,0.01,0.21,0.04,0.37
Blue,0.03,0.16,0.14,0.03,0.36
Hazel,0.03,0.02,0.09,0.02,0.16
Green,0.01,0.03,0.05,0.02,0.11
MarginalHair,0.18,0.22,0.49,0.11,1.00


In many cases we really only want to know the maximum of the marginal probability. For example, we can find the most probable eye color:

In [9]:
eyeHair.iloc[:3,4].idxmax()

'Brown'

While brown eyes are  the most probable, blue eyes are nearly as likely. 

Likewise, we can find the most probable hair color.  

In [10]:
eyeHair.iloc[4,:3].idxmax()

'Brunette'

## More on Bayes' theorem

We have already compute the probability of eye color given hair color in a previous section. Let's check that we get the same result using Bayes' theorem. This is another form of inference. 

By the chain rule of probabilities the joint distribution,  $P(hair,eye)$ , can be factored as:

$$P(hair,eye) = P(hair|eye)\ P(eye)\\
Or,
P(hair|eye) = \frac{P(hair,eye)}{P(eye)}$$

Now using Bayes' rule, we can write:

$$P(eye | hair) = \frac{P(hair|eye)\ P(eye)}{P(hair)} = \frac{P(hair,eye)\ P(eye)}{P(hair) \ P(eye)} = \frac{P(hair,eye)}{P(hair)}$$

The quantities $P(eye)$ and $P(hair)$ are just the marginal distributions. So, we can easily compute the conditional distribution of eye color give black hair:

In [11]:
PEyeGivenHair_black = eyeHair.loc[:,'Black'][:4]/eyeHair.loc['MarginalHair','Black']
PEyeGivenHair_black

Brown    0.611111
Blue     0.166667
Hazel    0.166667
Green    0.055556
Name: Black, dtype: float64

Finally, we can check that the result is a proper distribution by verifying the sum adds to 1.0. 

In [12]:
print('Sum of probabilites of eye given black hair = %4.2f' % sum(PEyeGivenHair_black)) 

Sum of probabilites of eye given black hair = 1.00


The above can be summarized by rewriting Bayes'theorem as follows:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}\\
= \frac{P(B|A)P(A)}{\sum_A P(A,B)}\\
= \frac{P(B|A)P(A)}{\sum_A P(B|A)\ P(A)}\\
= \frac{P(B|A)P(A)}{Marginal\ distribution\ of\ B}$$

In summry we can write the demominator in Bayes' theorm as a marginal distribution. 

## Independence and dependence

The **independence** structure is required to fully specify a probability distribution. Further, being able to specify an independence structure can greatly simplify the factorization of a distribution. On the other hand, **dependence** limits the ability to factorize a distribution.    

If $A$ and $B$ are independent then:

$$P(A|B) = P(A) \Leftrightarrow P(B|A) = P(B)$$

Consider the joint distribution $P(A,B)$. If $A$ is independent of $B$ then we can apply the chain rule of probability:

$$P(A,B) = P(A|B)P(B) = P(B|A)P(A) = P(A)P(B)$$

With the above relationships in mind, let's revisit the chain rule of probability: 

$$P(A_1, A_2, A_3, A_4 \ldots, A_n) = P(A_1 | A_2, A_3, A_4, \ldots, A_n)\ P(A_2 | A_3, A_4 \ldots, A_n)\ P(A_3| A_4 \ldots, A_n) \ldots p(A_n)$$

If all of $A_1, A_2, A_3, A_4 \ldots, A_n$ are all independent, then the chain rule of probability becomes:  

$$P(A_1, A_2, A_3, A_4 \ldots, A_n) = P(A_1)\ P(A_2)\ P(A_3) \ldots p(A_n)$$

This factorization leads to the **naive Bayes model**. The model is considered *naive*, since no dependency between the variables is accounted for.   

As another example consider $P(A,B,C,D)$ where 
- $A$ is dependent on $C$,
- $B$ is dependent on $D$,
- $A$ is independent of $B$ and $D$, 
- $B$ is independent of $C$, and
- $C$ is independent of $D$. 

We can write these **conditional independence** relationships as:

$$P(A \bot B,D| C) \\
P(B \bot A, C| D) \\
P(C \bot D)$$

In words, some variables are independent of other given the dependence on other. The symbol $\bot$ indicates independence. 

Using these conditional independences we can simplify the factorization of the distribution as follows:

$$P(A,B,C,D) = P(A|B,C,D)\ P(B|C,D)\ P(C|D)\ P(D) = P(A|C)\ P(B|D)\ P(C)\ P(D)$$

Notice how this independence/dependence structure simplifies the factorization of the distribution.  

## Law of total probability

The **law of total probability** is another useful tool for factorizing probability distributions. You can think of the law of total probability an extension of the idea of marginal distributions. 

Let's start with a conditional distribution $P(A|B,C)$ where $B \bot C$. Then we can factorize and simplify the distribution as:

$$P(A\ |\ C) = \sum_B P(A\ |\ B,C)\ P(B\ |\ C)$$

## Another example

As she approaches graduation a student is searching for a job as a machine learning engineer. In order to get the job she is interested in she must submit both GRE scores and a letter of recommendation from her machine learning professor. Unfortunately, the professor has a poor memory and will base his letter only on the grade the student received.  

The student would like to compute the joint distribution $P(D,I,S,G,L)$. She can make the following **assertions** about the dependencies and independencies for this problem. 

1. The degree of difficulty of the machine learning course is **unconditionally independent** of all other variable, $\{D \bot I,S,G.L \}$.
2. The intelligence of the student is **unconditionally independent** of all other variable, $\{I \bot D,S,G.L \}$.
3. The student's letter is **conditionally independent**  of intelligence, her GRE score, and her letter, given her grade, $\{L \bot I,S,L\ |\ G\}$.
4. The student's grade is **conditionally independent** of her GRE score and her letter, give the difficulty of the course and her intelligence, $\{G  \bot S,L\ |\ I,D\}$. 
5. The students GRE scores are **conditionally independent** of her grade, difficulty of the course and letter, given her intelligence, $\{S \bot G,D,L\ |\ I\}$.

Let's factorize the distribution, keeping in mind that the factorization is not unique. Given the assertions we find:

$$P(L,G,S,I,D) = P(D)\ P(I)\ P(S|I)\ P(G|D,I)\ P(L|G)$$


Let's assign some **prior probabilities** to these distributions, Starting with intelligence, I, which is independent of all other variables:

In [13]:
I = pd.DataFrame({'I':[0.2,0.8]}, index = ['I0','I1'])
I.transpose()

,I0,I1
I,0.2,0.8


The difficulty of the course is independent of all of variables as well:

In [14]:
D = pd.DataFrame({'D':[0.3,0.7]}, index = ['D0','D1'])
D.transpose()

,D0,D1
D,0.3,0.7


The GRE score, given intelligence, is a 2x2 table:

In [15]:
GREGivenI = pd.DataFrame({'S0':[0.95,0.1],'S1':[0.05,0.9]}, index = ['I0','I1'])
GREGivenI

,S0,S1
I0,0.95,0.05
I1,0.10,0.90


The distribution of the letter given the grade is a 2d, 3x2 column:

In [16]:
L = pd.DataFrame({'L0':[0.1,0.4,0.99],'L1':[0.9,0.6,0.01]}, index = ['G1','G2','G3'])
L

,L0,L1
G1,0.10,0.90
G2,0.40,0.60
G3,0.99,0.01


The situation with the course grade is a bit more complicated. The student's grade is dependent on both her intelligence and the difficulty of the course. We can represent this distribution using a Pandas `MultiIndex`: 

In [17]:
idx = [['I0','I0','I1','I1'], ['D0','D1','D0','D1']]
idx_tuple = list(zip(*idx))
index = pd.MultiIndex.from_tuples(idx_tuple, names=['I', 'D'])
print(index)
G = pd.DataFrame({'G1':[0.3,0.05,0.9,0.5],'G2':[0.4,0.25,0.08,0.3],'G3':[0.3,0.7,0.02,0.2]}, 
                 index = index)
G

MultiIndex(levels=[['I0', 'I1'], ['D0', 'D1']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['I', 'D'])


G1    G2    G3
I  D                   
I0 D0  0.30  0.40  0.30
   D1  0.05  0.25  0.70
I1 D0  0.90  0.08  0.02
   D1  0.50  0.30  0.20

In all of the above, notice that the rows add to 1.0. This must be the case for these representations to be valid joint distributions.   

The joint distributions can be computed by multiplication of these variables. For example, we can multiply the GRE score given intelligence by probability of intelligence:  

In [18]:
GREGivenI.mul(I.iloc[:,0], axis=0)

,S0,S1
I0,0.19,0.01
I1,0.08,0.72


You can see that these probabilities add up to 1.0. For this student with a high probability of being intelligent the most likely outcome is a good letter. 

Let's introduce some **evidence** into this problem. Let's say we know for a fact that the student is intelligent. Now we can easily compute the marginal distribution of her GRE score, which is just a row of the table:

In [19]:
GREGivenI.loc['I1']

S0    0.1
S1    0.9
Name: I1, dtype: float64

So, our student has a high probability of getting a good GRE score. 

Let's say that the machine learning course is, not too surprisingly, known to be difficult, another piece of **evidence**. We can compute the marginal probability of the students grade given these two bits of evidence:

In [20]:
G.loc['I1','D1']

G1    0.5
G2    0.3
G3    0.2
Name: (I1, D1), dtype: float64

Our student has a 50% probability of getting a top grade, even in the difficult course. 

What the student really wants to know is if she will get a good letter or not. We can compute the joint distribution of letter and grade:

In [21]:
DistLetter = L.transpose() * G.loc['I1','D1']
DistLetter

,G1,G2,G3
L0,0.05,0.12,0.198
L1,0.45,0.18,0.002


What the student really wants to know is the marginal distribution of letter, which is now easily calculated:

In [22]:
DistLetter.sum(axis = 1)

L0    0.368
L1    0.632
dtype: float64

In summary, our student has a good chance of getting a high GRE score (0.9) and a reasonable chance of getting a good letter (0.63). 

## One more example

The following example shows how we can expand the numerator and denominator when applying Bayes' theorem. The critical, and often difficult, part is to determine the probabilities of all of the alternative hypothesis summed to compute the numerator. 

Inspector Andrea Markov receives a call that Lord Fauntleroy has been murdered in his manor house. She is told that only the cook and butler were present in the house at the time of the crime and that there is no evidence of an intruder. Further, she learns that the only possible murder weapons are a knife and poison. 

The Inspector realizes that the cook and the butler could have acted alone or together Therefore, we will assume independence: $p(C,B) = p(C)p(B)$, where $p(C)$ is the probability the cook is guilty and $p(B)$ is the probability the butler is guilty.   

How can Inspector Markov compute a prior probability of the four possible cases, $p(B|K)$, $p(B|P)$, $p(C|K)$ and $p(C|P)$, where $K = knife$ and $P = poison$. Let's start with Bayes' Theorem for the first case:

$$p(B|K) = \frac{p(K|B)\ p(B)}{p(K)}$$

We can reformulate this relationship as the calculation of a marginal distribution:
$$p(B|K) = \sum_C p(B,C|K) \\
or,\\
= \sum_C \frac{p(B,C,K)}{P(k)} \\
= \frac{\sum_C p(K|B,C)\ P(B,C)}{\sum_{C,B,W} p(W|B,C)\ P(B,C)}$$

Notice that the numerator or normalization is summed over all variables to compute the **marginal distribution** of a knife being the murder weapon, $p(K)$.     

Now, keeping in mind that $p(C,B) = p(C)p(B)$ we can write this relationship:

$$p(B|K) = \frac{p(B)\ \sum_C p(K|B,C)\ P(C)}{\sum_B P(B)\ \sum_{C,W} p(W|B,C)\ P(C)} \\
= \frac{p(B)\ \sum_C p(K|B,C)\ P(C)}{\sum_B P(B)\ \sum_C P(C)\ \sum_{W} p(W|B,C)}$$

The factorization of the numerator is not unique. We can also write:

$$p(B|K)
= \frac{p(B)\ \sum_C p(K|B,C)\ P(C)}{\sum_C P(C)\ \sum_B P(B)\ \sum_{W} p(W|B,C)}$$

So, the numerator is the same for both the case of the butler or the cook being the murderer. 

Before we expand this relationship, we need to introduce some notation. Let $\wedge$ be the logical AND operator and $\urcorner$ be the logical NOT operator. Using this notation we can expand the numerator:

$$p(B)\ \sum_C p(K|B,C)\ P(C) \\
= p(B) \big[ p(C)\ p(K|B \wedge C) +  p( \urcorner C)\ p(K|B \wedge \urcorner C) \big]$$

This does not look too bad. 

The denominator is quite a bit more complicated as there are 8 possible cases. 

$$\sum_B P(B)\ \sum_C P(C)\ \sum_{W} p(W|B,C) $$   
   
Now, the probability of poison has been determined to be zero; $p(P) = 0$. This reduces the number of possible cases to 8.  Additionally, since there are only two possible suspects, $p(\urcorner B \wedge \urcorner C) = 0$. The two cases (one for each possible weapon) where neither butler or cook committed the crime must be zero. We can now expand the denominator with these simplifications as follows:
    
$$= p(B) \Big[ p(C)\ \big( p(K|B \wedge C) + p(P|B \wedge C)\big)  + p( \urcorner C)\ \big(  p(K|B \wedge \urcorner C) + p(P|B \wedge \urcorner C)  \big) \Big] \\
+ p(\urcorner B) \Big[ p(C)\ \big( p(K| \urcorner B \wedge C) + p(P| \urcorner B \wedge C) \big) \Big]$$

In summary, the denominator sums the probabilities of all possible alternative hypothesis. Details for the other cases are left to you as an exercise. 

As advertised there are six none-zero probability alternatives in the denominator. Consider what would happen if some the variables had more than two possible states. For example, how many alternatives are there if there were four possible murder weapons. Or, how many states are created if we added 2 addition suspects as well as four possible murder weapons. These though experiments should lead you to understand that computational complexity grows rather quickly (combinatorically) with the number of states.   

#### Copyright 2018, Stephen F. Elston. All rights reserved.